In [67]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import random
import re
import time
import plotly.graph_objects as go
import plotly.express as px
import itertools

**Conjuntos**

$$ \text{CLIENTES} = \{1, 2, \ldots, N\} $$
$$ \text{NODOS} = \{0\} \cup \text{CLIENTES} $$
$$ \text{CAMIONES} = \{1, 2, 3, 4, 5, 6, 7\} $$

In [68]:
N = 20
CLIENTES = list(range(1, N + 1))
NODOS = [0] + CLIENTES
CAMIONES = [1, 2, 3, 4, 5]

In [69]:
random.seed(142)

**Parámetros**

$$ \text{CAPACIDAD\_CAMION}_c = 70 \text{ para cada camión } c \in \text{CAMIONES} $$
$$ \text{DEMANDA}_{cl}: \text{Demanda del cliente } cl \in \text{CLIENTES} $$
$$ \text{DISTANCIAS}_{n1,n2}: \text{Distancia euclidiana entre } n1 \in \text{NODOS} \text{ y } n2 \in \text{NODOS} $$

In [70]:
CAPACIDAD_CAMION = {c: 70 for c in CAMIONES}
DEMANDAS = {c: random.randint(10, 20) for c in CLIENTES}
COORDENADAS = {n: {"x": random.randint(0, N * 10), "y": random.randint(0, N * 10)} for n in NODOS}
DISTANCIAS = {(n1, n2): round(((COORDENADAS[n1]["x"] - COORDENADAS[n2]["x"]) ** 2 + (COORDENADAS[n1]["y"] - COORDENADAS[n2]["y"]) ** 2) ** 0.5) for n1 in NODOS for n2 in NODOS}

In [71]:
fig = go.Figure()

# Añadir punto de depósito en rojo
fig.add_trace(
    go.Scatter(
        x=[COORDENADAS[0]["x"]],
        y=[COORDENADAS[0]["y"]],
        mode="markers",
        name="Depósito",
        marker=dict(color="red", size=15),
    )
)

# Añadir puntos de clientes en negro
fig.add_trace(
    go.Scatter(
        x=[coord["x"] for coord in list(COORDENADAS.values())[1:]],
        y=[coord["y"] for coord in list(COORDENADAS.values())[1:]],
        mode="markers",
        name="Clientes",
        marker=dict(color="black", size=10),
    )
)

# Añadir anotaciones de clientes
for cliente in CLIENTES:
    coord_cliente_x = COORDENADAS[cliente]["x"]
    coord_cliente_y = COORDENADAS[cliente]["y"]
    fig.add_annotation(x=coord_cliente_x - 8, y=coord_cliente_y + 8, text=f"<b>C{cliente}: </b> {DEMANDAS[cliente]}", showarrow=False)

# Configurar el layout del gráfico
fig.update_layout(
    title=f"COORDENADAS NODOS",
    xaxis_title="X",
    yaxis_title="Y",
    xaxis=dict(range=[-10, N * 10 + 10]),
    yaxis=dict(range=[-10, N * 10 + 10]),
    width=800,
    height=600,
    template="ggplot2",
)

fig.show()

In [72]:
# Crear el problema de enrutamiento
manager = pywrapcp.RoutingIndexManager(len(NODOS), len(CAMIONES), 0)
routing = pywrapcp.RoutingModel(manager)

In [73]:
# Crear y registrar una función de callback para las distancias
def distancia_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return DISTANCIAS[from_node, to_node]


transit_callback_index = routing.RegisterTransitCallback(distancia_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


# Agregar restricción de capacidad
def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return DEMANDAS.get(from_node, 0)


demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, [CAPACIDAD_CAMION[c] for c in CAMIONES], True, "Capacity")  # slack  # capacidades de vehículos  # start cumul to zero


True

In [74]:
# Configuración de parámetros de búsqueda
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
search_parameters.time_limit.seconds = 30

In [75]:
# Resolver el problema
solution = routing.SolveWithParameters(search_parameters)

In [76]:
if solution:
    total_distance = 0
    for vehicle_id in range(len(CAMIONES)):
        index = routing.Start(vehicle_id)
        plan_output = "Route for vehicle {}:\n".format(CAMIONES[vehicle_id])
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += " {} ->".format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += " {}\n".format(manager.IndexToNode(index))
        plan_output += "Distance of the route: {}m\n".format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print("Total Distance of all routes: {}m".format(total_distance))
else:
    print("No solution found!")


Route for vehicle 1:
 0 -> 8 -> 6 -> 10 -> 17 -> 0
Distance of the route: 230m

Route for vehicle 2:
 0 -> 9 -> 11 -> 5 -> 15 -> 14 -> 0
Distance of the route: 339m

Route for vehicle 3:
 0 -> 12 -> 1 -> 19 -> 2 -> 0
Distance of the route: 342m

Route for vehicle 4:
 0 -> 7 -> 3 -> 18 -> 0
Distance of the route: 148m

Route for vehicle 5:
 0 -> 4 -> 20 -> 13 -> 16 -> 0
Distance of the route: 372m

Total Distance of all routes: 1431m


In [77]:
rutas = {}
if solution:
    total_distance = 0
    for vehicle_id in range(len(CAMIONES)):
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        total_distance += route_distance
        route.append(manager.IndexToNode(index))
        rutas[CAMIONES[vehicle_id]] = route

In [78]:
fo = sum(sum([DISTANCIAS[(ruta[i], ruta[i + 1])] for i in range(len(ruta) - 1)]) for ruta in rutas.values())

In [79]:
fig = go.Figure()

# RUTAS
for id_camion, ruta in rutas.items():
    coords_x = [COORDENADAS[nodo]["x"] for nodo in ruta]
    coords_y = [COORDENADAS[nodo]["y"] for nodo in ruta]
    capacidad_ruta = sum(map(lambda x: DEMANDAS[x] if x != 0 else 0, ruta))
    distancia_ruta = sum([DISTANCIAS[(ruta[i], ruta[i + 1])] for i in range(len(ruta) - 1)])

    fig.add_trace(
        go.Scatter(
            x=coords_x,
            y=coords_y,
            mode="lines",
            line=dict(color=px.colors.qualitative.G10[id_camion % len(px.colors.qualitative.G10)], width=2, dash="solid"),
            name=f"Ruta C{id_camion} - CAP: {capacidad_ruta} D: {distancia_ruta:.2f}",
        )
    )

    # FLECHAS
    for i in range(len(ruta) - 1):
        mid_x = (coords_x[i] + coords_x[i + 1]) / 2
        mid_y = (coords_y[i] + coords_y[i + 1]) / 2
        fig.add_annotation(
            x=mid_x, y=mid_y,
            ax=mid_x - (coords_x[i + 1] - coords_x[i]) * 0.1, ay=mid_y - (coords_y[i + 1] - coords_y[i]) * 0.1,
            xref="x",
            yref="y",
            axref="x",
            ayref="y",
            showarrow=True,
            arrowhead=1,
            arrowsize=2,
            arrowwidth=1,
            arrowcolor=px.colors.qualitative.G10[id_camion % len(px.colors.qualitative.G10)],
        )

# DEPÓSITO
fig.add_trace(
    go.Scatter(
        x=[COORDENADAS[0]["x"]],
        y=[COORDENADAS[0]["y"]],
        mode="markers",
        name="Depósito",
        marker=dict(color="red", size=15),
    )
)

# CLIENTES
fig.add_trace(
    go.Scatter(
        x=[coord["x"] for coord in list(COORDENADAS.values())[1:]],
        y=[coord["y"] for coord in list(COORDENADAS.values())[1:]],
        mode="markers",
        name="Clientes",
        marker=dict(color="black", size=10),
    )
)

for cliente in CLIENTES:
    coord_cliente_x = COORDENADAS[cliente]["x"]
    coord_cliente_y = COORDENADAS[cliente]["y"]
    fig.add_annotation(x=coord_cliente_x - 8, y=coord_cliente_y + 8, text=f"<b>C{cliente}: </b> {DEMANDAS[cliente]}", showarrow=False)


fig.update_layout(
    title=f"RECORRIDO TOTAL: {fo:.2f}",
    xaxis_title="X",
    yaxis_title="Y",
    xaxis=dict(range=[-10, N * 10 + 10]),
    yaxis=dict(range=[-10, N * 10 + 10]),
    width=800,
    height=600,
    template="ggplot2",
)

fig.show()